# Network Sampling ABM

Provides new class partly based on code from <i>Random Walk</i> but now provides functionality to model many sampling methods besides random walk.

## Network Sampling Functions

Note that each sampling method simply returns the next chosen node to visit for an agent.

In [2]:

from __future__ import annotations

def random_walk(agent: NXAgent):
    """Random Walk Algorithm"""
    nt = agent.get_network()
    neighbors = list(nx.neighbors(G=nt, n=agent.node))
    new_node = np.random.choice(a=neighbors, size=None)
    return new_node
    
    
def random_walk_weighted(agent: NXAgent):
    """Random Walk Algorithm chooses next vertex based on percentage of vertex weight in neighborhood"""
    nt = model.network
    neighbors = list(nx.neighbors(G=nt, n=agent.node))
    weights = []
    for n in neighbors:
        """Generate list of weights for nodes"""
        if 'weight' in n:
            weights.append(float(n['weight']))
        else:
            weights.append(float(0))

    new_node = np.random.choice(a=neighbors, p=weights)
    return new_node
        

## Agent and Model Classes

In [5]:

from __future__ import annotations
from inspect import signature
from mesa import Agent, Model
from mesa.time import SimultaneousActivation
import networkx as nx
import numpy as np
from typing import Any, Callable, Dict

class NXAgent(Agent):
    """Agent integrated with networkx"""    
    def __init__(self, unique_id: int, model: NSModel, node) -> None:
        """
        Initializes required attributes under Agent
        
        Parameters
        unique_id - unique id inherited from mesa.Agent
        model - model inherited from mesa.Model
        node - current node NXAgent is occupying in model
        """
        super().__init__(unique_id=unique_id, model=model)
        try:
            if node not in model.network.nodes:
                raise ValueError('node not in model\'s network')
        except ValueError as error:
            print(str(error))
            del self
        self.extra_properties = Dict[str, Any]
        self.node = node
        self.visited = np.asarray(a=[node])
    
    @property
    def algorithm(self) -> Callable:
        """Get network sampling method employed at each step of ABM"""
        return self.__algorithm
    
    @algorithm.setter
    def algorithm(self, new_algorithm: Callable) -> None:
        """Sets new algorithm / smapling method for agent"""
        # Error checking for correct function signature
        sig = signature(new_algorithm)
        try:
            if len(sig.parameters) != 1:
                raise ValueError('new_algorithm must have one parameter')
        except ValueError as error:
            print(error)
            return
        self.__algorithm = new_algorithm
        self.__visited_nodes = np.asarray(a=[])
    
    @property
    def node(self) -> object:
        """Current node or vertex NXAgent owns"""
        return self.__node
    
    @node.setter
    def node(self, new_node) -> None:
        """
        Sets new node or vertex for NXAgent to own
        
        Parameters
        new_node - new node for current NXAgent object to be located at
        """
        # Error checking for valid new node (existing in model) 
        try:
            if not new_node in self.get_network():
                raise ValueError('new node must be present in current model\'s network')
        except ValueError as error:
            print(str(error))
            return
        self.__node = new_node
    
    @property
    def visited_nodes(self) -> np.ndarray:
        """Array of visited nodes"""
        return self.__visited_nodes
    
    @visited_nodes.setter
    def visited_nodes(self, new_visited_nodes: np.ndarray) -> None:
        """Sets new history of visited nodes"""
        self.__visited_nodes = new_visited_nodes
    
    # ACCESSORS
    def get_network(self) -> nx.Graph:
        """Gets Networkx object, ie the network to be used in the model"""
        return self.model.network
    
    # MUTATORS
    def clear_visited_nodes(self) -> None:
        """Clears history of visited nodes"""
        self.__visited_nodes.clear()
    
    def step(self):
        """What the agent does at each step of ABM"""
        new_node = self.algorithm(agent=self)
        self.node = new_node
        self.visited_nodes = np.append(arr=self.visited_nodes, values=new_node)
      
# NS = Network Sampling
class NSModel(Model):
    """Model integrated with networkx and base class for random walks"""
    def __init__(self, algorithm: Callable, network: nx.Graph, num_agents: int, start_node) -> None:
        """
        Initializes base network
        
        Parameters
        algorithm - graph sampling method function, taking only two parameters (model and agent)
        network - nx.Graph object model is based on
        num_agents - number of NXAgent objects to add to schedule
        start_node - node where all NXAgent objects initially reside
        """
        super().__init__()
        
        # Checking for non-empty graph with nodes
        try:
            if nx.is_empty(G=network):
                raise ValueError('network does not contain any nodes')
            if num_agents < 0:
                raise ValueError('number of agents cannot be negative')
            if start_node not in network.nodes:
                raise ValueError('start node is not in network')
        except ValueError as error:
            print(str(error))
            del self
            
        # Building up network for model
        self.network = network
        self.start_node = start_node
        self.schedule = SimultaneousActivation(model=self)
        self.agents = np.asarray(a=[], dtype=NXAgent)
        for id in np.arange(num_agents):
            a = NXAgent(unique_id=id, model=self, node=start_node)
            a.algorithm = algorithm
            self.schedule.add(agent=a)
            self.agents = np.append(arr=self.agents, values=a)

    @property
    def agents(self) -> np.ndarray:
        """Returns all NXAgents aboard the model"""
        return self.__agents
    
    @agents.setter
    def agents(self, new_agents: np.ndarray) -> None:
        """
        Setter for agents
        
        Parameters
        new_agents - numpy array of new NXAgent objects to repkace existing NXAgent objects
        """
        # Error checking for agent node validity
        try:
            if not np.all(a=[agent.node in self.network.nodes for agent in new_agents]):
                raise ValueError('Each element in new_agents must possess a node in current network')
        except ValueError as error:
            print(str(error))
            return
        self.__agents = new_agents

    @property
    def network(self) -> nx.Graph:
        """Base network holding the model"""
        return self.__network
    
    @network.setter
    def network(self, new_network: nx.Graph) -> None:
        """
        Sets new network for model
        
        Parameters
        new_network - new networkx graph for NXAgent objects to traverse through as model
        """
        self.__network = new_network
    
    @property
    def number_of_agents(self) -> int:
        """Count of NXAgents used by the model"""
        return self.agents.size
    
    @property
    def start_node(self):
        """Gets initialized node that all agents start at"""
        return self.__start_node
    
    @start_node.setter
    def start_node(self, new_start_node) -> None:
        """Resets start node for model"""
        try:
            if new_start_node not in self.network.nodes:
                raise ValueError('new_start_node is not in current network')
        except ValueError as error:
            print(str(error))
            return
        self.__start_node = new_start_node
    
    # MUTATORS
    def reset(self) -> None:
        """Resets all NXAgents back to start_node with cleared visit history"""
        for agent in self.__agents:
            agent.clear_visited_nodes()
            agent.node = self.__start_node
        
    def step(self, num_steps: int) -> None:
        """
        Activates model to run n steps for each NXAgent
        
        Parameters
        num_steps - number of steps for each NXAgent to step through
        """
        for step_number in np.arange(num_steps):
            self.schedule.step()


## Testing NSModel

In [6]:

import networkx as nx

nsmodel = NSModel(algorithm=random_walk, network=nx.cycle_graph(n=20), num_agents=5, start_node=0)
nsmodel.step(num_steps=5)
for agent in nsmodel.agents:
    """Prints out journey of each agent"""
    print(f'Agent: {agent.unique_id}, visited_nodes: {agent.visited_nodes}')


Agent: 0, visited_nodes: [1. 2. 3. 2. 1.]
Agent: 1, visited_nodes: [19.  0.  1.  2.  3.]
Agent: 2, visited_nodes: [1. 0. 1. 0. 1.]
Agent: 3, visited_nodes: [19.  0. 19. 18. 17.]
Agent: 4, visited_nodes: [1. 0. 1. 2. 1.]


## NSTest

Complete testing framework for sampling simulation model, evaluated by an agent-based model.

In [ ]:

from collections import namedtuple
import param


class NSTest:
    """Framework creates, executes, and evaluates performance on network sampling algorithms"""
    def __init__(self, num_trials: int) -> None:
        """Initializes"""
        self.num_trials = Param.Integer()

